<a href="https://colab.research.google.com/github/gmauricio-toledo/NLP-LCC/blob/main/Rest-Mex/Extender_vectores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Rest Mex 2025</h1>
<h3>Vectores mixtos: concatenando embeddings y conteos</h3>

**Actividad por equipo**. Cada equipo debe presentar 3 diferentes experimentos:

1. Clasificación usando los vectores aumentados, usando el preprocesamiento y estrategia de balanceo que han estado usando en los experimentos, reportar el score F1. Para la parte de embeddings usar `doc2vec`.

2. Clasificación usando los vectores aumentados, usando el preprocesamiento y estrategia de balanceo que han estado usando en los experimentos, reportar el score F1. Para la parte de embeddings usar promedios de `word2vec`/`fasttext`/`glove`.

3. En el modelo que haya funcionado mejor de los dos anteriores, presentar otro experimento variando la longitud del vector de conteos tf-idf.

Es posible que se tengan que hacer estos *downgrades*

In [ ]:
!pip uninstall -y numpy scipy gensim
!pip install numpy==1.24.3
!pip install scipy==1.10.1
!pip install gensim==4.3.1

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scipy 1.14.1
Uninstalling scipy-1.14.1:
  Successfully uninstalled scipy-1.14.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 25.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hyperopt 0.2.7 requires scipy, which is not installed.
sklearn-pandas 2.2.0 requires scipy>=1.5.1, which is not installed.
pytensor 2.30.3 requires scipy<2,>=1, which is not installed.
yellowbrick 1.5 requires scipy>=1.0.0, which is not installed.
hdbscan 0.8.40 requires scipy>=1.0, which is not installed.
mlxtend 0.23.4 requires scipy>=1.2.1, which is not installed.
mizani 0.13.3 requires scipy>=1.8.0, which is not installed.
shap 0.47.1 requires scipy, which is not installed.
fastai 2.7.19 requires scipy, which is not insta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 15.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cvxpy 1.6.5 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
scikit-image 0.25.2 requires scipy>=1.11.4, but you have scipy 1.10.1 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.

Más información: https://sites.google.com/cimat.mx/rest-mex-2025/home

In [ ]:
import re
import nltk
from nltk import word_tokenize
from nltk import word_tokenize
from string import punctuation

nltk.download('punkt_tab')
nltk.download('stopwords')

stopwords_list = nltk.corpus.stopwords.words('spanish')
punct_set = set(punctuation)

def clean_document(doc):
   pass

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/gmauricio-toledo/NLP-LCC/main/Rest-Mex/Rest-Mex_2025_train.csv"
df = pd.read_csv(url)
df.head()

,Title,Review,Polarity,Town,Region,Type
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,5.0,Sayulita,Nayarit,Restaurant
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",4.0,Tulum,QuintanaRoo,Attractive
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",3.0,Tulum,QuintanaRoo,Hotel
3,un buen panorama cerca de CancÃºn,"Estando en CancÃºn, fuimos al puerto y tomamos...",4.0,Isla_Mujeres,QuintanaRoo,Attractive
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,5.0,Patzcuaro,Michoacan,Hotel


#1. Polaridad

In [ ]:
df1 = df[['Title','Review','Region','Polarity']]
df1

,Title,Review,Region,Polarity
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,Nayarit,5.0
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",QuintanaRoo,4.0
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",QuintanaRoo,3.0
3,un buen panorama cerca de CancÃºn,"Estando en CancÃºn, fuimos al puerto y tomamos...",QuintanaRoo,4.0
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,Michoacan,5.0
...,...,...,...,...
208046,"excelente ambiente, comida y atencion","Excelente Restaurante, comida Mexicana de alto...",Queretaro,5.0
208047,Muy mal servicio,Estuvimos allí como huéspedes de un día a la p...,QuintanaRoo,2.0
208048,Excelente,"Excelente comida, así como la atención y servi...",Estado_de_Mexico,5.0
208049,Visita nocturna,Nos toco visitar este sitio cuando ya había ca...,Chihuahua,4.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


df1['Text'] = df1['Title'] + ' ' + df1['Review']
df1.dropna(subset=['Text'], inplace=True)

X = df1['Text'].values

y = df1['Polarity']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y.astype(int),
                                                    test_size=0.2,
                                                    random_state=2025)

train_docs = [clean_document(doc) for doc in X_train]
test_docs = [clean_document(doc) for doc in X_test]

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(train_docs)
X_test_tfidf = vectorizer.transform(test_docs)

<ipython-input-4-cf0b8a4b4df4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Text'] = df1['Title'] + ' ' + df1['Review']
<ipython-input-4-cf0b8a4b4df4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.dropna(subset=['Text'], inplace=True)


In [ ]:
import numpy as np

vocab = vectorizer.get_feature_names_out()
important_words = []

for label in np.unique(y_train):
    X_j = X_train_tfidf[y_train==label].copy()
    normas = np.sum(np.abs(X_j), axis=0)
    word_normas = [(word, norma) for word, norma in zip(vocab, normas.tolist()[0])]
    word_normas = sorted(word_normas, key=lambda x: x[1], reverse=True)[:5]
    print(f"Label: {label}")
    for word, norma in word_normas:
        print(word,end=' ')
    print()
    important_words.extend([word for word, norma in word_normas])

Label: 0
servicio mal simo hotel mala 
Label: 1
servicio comida hotel mal lugar 
Label: 2
comida lugar est hotel bien 
Label: 3
lugar buena comida buen hotel 
Label: 4
lugar excelente comida servicio mejor 


In [ ]:
important_words = list(set(important_words))
print(important_words)
important_words_idxs = [vocab.tolist().index(word) for word in important_words]
print(important_words_idxs)

['lugar', 'mejor', 'est', 'excelente', 'buena', 'mal', 'hotel', 'servicio', 'comida', 'mala', 'simo', 'bien', 'buen']
[52417, 54962, 34216, 35287, 12591, 53061, 43696, 78447, 19027, 53062, 79162, 10996, 12590]


In [ ]:
extra_train_features = X_train_tfidf[:, important_words_idxs].toarray()
print(extra_train_features.shape)
extra_test_features = X_test_tfidf[:, important_words_idxs].toarray()
print(extra_test_features.shape)

(166439, 13)
(41610, 13)


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

tokenized_train_docs = [word_tokenize(doc) for doc in train_docs]
tokenized_test_docs = [word_tokenize(doc) for doc in test_docs]

tagged_data = [TaggedDocument(words=doc, tags=[str(i)])
               for i, doc in enumerate(tokenized_train_docs)]

# Parametros
vector_size = 200   # Dimensionality of the feature vectors
window = 5        # Maximum distance between current and predicted word within a sentence
min_count = 5     # Ignores all words with total frequency lower than this
workers = -1       # Use `workers` number of worker threads to train the model
epochs = 5       # Number of iterations (epochs) over the corpus

d2v_model = Doc2Vec(vector_size=vector_size,
                 window=window,
                 min_count=min_count,
                 workers=workers)

d2v_model.build_vocab(tagged_data)

# Entrenamiento
d2v_model.train(tagged_data, total_examples=d2v_model.corpus_count, epochs=epochs)

In [ ]:
train_embeddings = d2v_model.dv.vectors
test_embeddings = np.array([d2v_model.infer_vector(doc) for doc in tokenized_test_docs])

augmented_train_features = np.concatenate((train_embeddings, extra_train_features), axis=1) # Esta es la matriz de vectores aumentados
print(augmented_train_features.shape,train_embeddings.shape,extra_train_features.shape)
augmented_test_features = np.concatenate((test_embeddings, extra_test_features), axis=1)
print(augmented_test_features.shape,test_embeddings.shape,extra_test_features.shape)

(166439, 213) (166439, 200) (166439, 13)
(41610, 213) (41610, 200) (41610, 13)


Ya puedes hacer el entrenamiento y evaluación con:

* Vectores aumentados: `augmented_train_features`
* Vectores sin aumentar (word2vec/glove/fasttext/doc2vec) `train_embeddings`

# 2. Tipo de lugar

In [ ]:
df2 = df[['Title','Review','Region','Type']]
df2

,Title,Review,Region,Type
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,Nayarit,Restaurant
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",QuintanaRoo,Attractive
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",QuintanaRoo,Hotel
3,un buen panorama cerca de CancÃºn,"Estando en CancÃºn, fuimos al puerto y tomamos...",QuintanaRoo,Attractive
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,Michoacan,Hotel
...,...,...,...,...
208046,"excelente ambiente, comida y atencion","Excelente Restaurante, comida Mexicana de alto...",Queretaro,Restaurant
208047,Muy mal servicio,Estuvimos allí como huéspedes de un día a la p...,QuintanaRoo,Hotel
208048,Excelente,"Excelente comida, así como la atención y servi...",Estado_de_Mexico,Restaurant
208049,Visita nocturna,Nos toco visitar este sitio cuando ya había ca...,Chihuahua,Attractive


#3. Pueblo

In [ ]:
df3 = df[['Title','Review','Region','Town']]
df3

,Title,Review,Region,Town
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,Nayarit,Sayulita
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",QuintanaRoo,Tulum
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",QuintanaRoo,Tulum
3,un buen panorama cerca de CancÃºn,"Estando en CancÃºn, fuimos al puerto y tomamos...",QuintanaRoo,Isla_Mujeres
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,Michoacan,Patzcuaro
...,...,...,...,...
208046,"excelente ambiente, comida y atencion","Excelente Restaurante, comida Mexicana de alto...",Queretaro,Tequisquiapan
208047,Muy mal servicio,Estuvimos allí como huéspedes de un día a la p...,QuintanaRoo,Tulum
208048,Excelente,"Excelente comida, así como la atención y servi...",Estado_de_Mexico,Ixtapan_de_la_Sal
208049,Visita nocturna,Nos toco visitar este sitio cuando ya había ca...,Chihuahua,Creel


# Test

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/gmauricio-toledo/NLP-LCC/main/Rest-Mex/Rest-Mex_2025_test.xlsx"
df = pd.read_excel(url)
df.drop(columns='ID',inplace=True)
df.head()

,Title,Review
0,Una de las mejores comidas en Sayulita,"Comida exelente, el lugar muy cómodo y persona..."
1,Me gustó mucho...,En mi camino hacia Mahahual paré a comer en es...
2,Excelentes restaurantes en hotel Guaycura,"Excelente Servicio, la comida es deliciosa tan..."
3,Gran desayuno y almuerzo,Mi esposa y yo fuimos allí dos veces. Una vez ...
4,Mejor atención de la zona hotelera!,"Increíble restaurant! La comida riquísima, la ..."
